In [6]:
# Steps in Face recognition:
# 1) Creating the Dataset
# 2) Training the Recognizer
# 3) Face Detection ->Detector

# Importing libraries

In [7]:
import cv2
import numpy
import sqlite3
import csv
import os

# Face capture from the image

In [8]:
face_detection = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)

# Function to insert the passenger details in the database

In [9]:
def insertUpdateDB(Id,name,age,gender,date):
    conn = sqlite3.connect("TicketFaceRecog.db")
    cmd="SELECT * FROM Reservation WHERE ID="+str(Id)
    cursor = conn.execute(cmd)
    exists=0
    for row in cursor:
        exists=1
    if(exists==1):
        cmd="UPDATE Reservation SET Name=' "+str(name)+" ' WHERE ID="+str(Id)
    else:
        cmd="INSERT INTO Reservation(ID,Name) Values("+str(Id)+",' "+str(name)+" ' )"
    conn.execute(cmd)
    conn.commit()
    conn.close()

#  Create/update the CSV file to add the data

In [10]:
columns = ['SERIAL NO.', '', 'PASSENGER UNIQUE ID', '', 'PASSENGER NAME', '', 'PASSENGER AGE', '','PASSENGER GENDER','', 'DATE_OF_TRAVEL']
serial = 0
exists = os.path.isfile("ReservationDetails.csv")
if exists:
        with open("ReservationDetails.csv", 'r', newline='') as csvFile1:
            reader1 = csv.reader(csvFile1)
            for l in reader1:
                serial = serial + 1
        serial = (serial // 2)
        csvFile1.close()
else:
        with open("ReservationDetails.csv", 'a+', newline='') as csvFile1:
            writer = csv.writer(csvFile1)
            writer.writerow(columns)
            serial = 1
        csvFile1.close()

# Passenger Detail Inputs (User input)

In [11]:
user_id = input("Enter the ID: ")
name = input("Enter the name: ")
age=input("Enter the age: ")
gender = input("Enter the gender(M or F): ")
date_of_journey = input("Enter the date: ")
row = [serial, '', user_id, '', name, '', age, '',gender,'', date_of_journey]

Enter the ID: 2008
Enter the name: Akash.U.N
Enter the age: 19
Enter the gender(M or F): M
Enter the date: 04-12-2021


# Insert the passenger details into the CSV File

In [12]:
with open('ReservationDetails.csv', 'a+', newline='') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
csvFile.close()
count = 0

# Insert the passenger details into the Database

In [13]:
insertUpdateDB(user_id,name,age,gender,date_of_journey)

# Face capture - Convert to Gray - Dataset folder with 0-20 images

In [14]:
while(True):
    ret, color_img = camera.read()
    gray_image = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray_image, 1.3, 5)

    for (x, y, w, h) in faces:
        count += 1
        cv2.imwrite("DataSet/User."+str(user_id)+"." +
                    str(count)+".jpg", gray_image[y:y+h, x:x+w])
        cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 0), 3)
        cv2.waitKey(100)

    cv2.imshow("Face Detection", color_img)
    cv2.waitKey(1)

    if(count > 20):
        break

camera.release()
cv2.destroyAllWindows()